In [ ]:
import copy
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns



Read stored data from the data_preparation notebook (that needs to have run beforehand).

In [ ]:
# @formatter:off
%store -r df_airports
%store -r airports_dict
%store -r df_merged
# @formatter:on
GG = nx.read_gml('Graphs/airlines.gml')


Calculate the centralities metrics for the network, and set them as node attributes, and for easier plotting,
we include the centralities in an array.

In [ ]:
degree_centrality = nx.degree_centrality(GG)
closeness_centrality = nx.closeness_centrality(GG)
betweenness_centrality = nx.betweenness_centrality(GG)
eigenvector_centrality = nx.eigenvector_centrality(GG)

nx.set_node_attributes(GG, closeness_centrality, 'closeness centrality')
nx.set_node_attributes(GG, degree_centrality, 'degree centrality')
nx.set_node_attributes(GG, betweenness_centrality, 'betweenness centrality')
nx.set_node_attributes(GG, eigenvector_centrality, 'eigenvector centrality')


centralityArr = [degree_centrality, closeness_centrality,
                 betweenness_centrality, eigenvector_centrality]
centralityNames = ['degree', 'closeness', 'betweenness', 'eigenvector']


We need to convert the airports long/lat positions into the map projection

c.f. https://matplotlib.org/basemap/users/mapcoords.html

In [ ]:
map = Basemap(projection='cyl', resolution=None,
              llcrnrlat=-90, urcrnrlat=90,
              llcrnrlon=-180, urcrnrlon=180, )

lats = [k[1]['Latitude'] for k in airports_dict.items()]
longs = [k[1]['Longitude'] for k in airports_dict.items()]

pos = {}
for n in GG.nodes():
    x,y = map(GG.nodes[n]['Longitude'], GG.nodes[n]['Latitude'])
    pos[n] = (x,y)

## Centralities
Plot each previously computed centrality on the world map.
These plots use the `viridis` colormap, c.f. https://matplotlib.org/stable/tutorials/colors/colormaps.html
where purple values are low, blue-greenish in the middle and high value are mapped to yellow.

In [ ]:
def normalize(x):
    """
    Returns normalized array with values in [0,1] range
    :param x:
    :return:
    """
    return (x - np.min(x)) / (np.max(x) - np.min(x))

In [ ]:
nodesizes = {
    'degree':50,
    'closeness':1,
    'betweenness':100,
    'eigenvector':35
}
fig, axes = plt.subplots(2, 2, figsize=(12,8))
for centrality, name, ax in zip(centralityArr, centralityNames, axes.flat):
    minCentralityVal = min(centrality.values())
    maxCentralityVal = max(centrality.values())
    print(name, minCentralityVal, maxCentralityVal)

    # normalize values into [0,1] for color map
    ff = np.array(list(nx.get_node_attributes(
        GG, f'{name} centrality').values()))
    # CC = normalize(ff)
    CC = ff

    # fig2, ax2 = plt.subplots(dpi=100, figsize=(4, 3))
    # fig2, ax2 = plt.subplots(dpi=1000, figsize=(8, 6))
    map = Basemap(projection='cyl', resolution=None,
                  llcrnrlat=-90, urcrnrlat=90,
                  llcrnrlon=-180, urcrnrlon=180, ax=ax )
    cmap = plt.cm.get_cmap('viridis')
    vmin = min(CC)
    vmax = max(CC)
    map.bluemarble(alpha=0.6)
    # draw twice: first one is lines only with alpha 0.1, second one is nodes only
    nx.draw_networkx(GG, pos=pos, node_size=0, width=0.0001, node_color=CC, vmin=vmin, vmax=vmax,
                     cmap=cmap, arrows=False, with_labels=False, edge_color='w', alpha=0.1,
                     node_shape='o', ax=ax)
    nx.draw_networkx(GG, pos=pos, width=0.0, node_color=CC, vmin=vmin, vmax=vmax,
                     cmap=cmap, arrows=False, with_labels=False, edge_color='w', alpha=0.8,
                     node_shape='o', node_size=[v * nodesizes[name] for v in centrality.values()], ax=ax)

    # Create colorbar according to https://stackoverflow.com/a/49572947/14720136
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

    sm = plt.cm.ScalarMappable(cmap=cmap)
    sm.set_array([])
    # custom ticks https://matplotlib.org/3.3.3/gallery/ticks_and_spines/colorbar_tick_labelling_demo.html
    cbar = plt.colorbar(sm, orientation='horizontal', aspect=50, ticks=[0,1], pad = 0.01, ax=ax)
    cbar.ax.set_xticklabels([f'{vmin:.2f}', f'{vmax:.2f}'])
    ax.set_title(f'{name.capitalize()} Centrality')

# plt.title(f'{name.capitalize()} Centrality', fontsize=20)
fig.suptitle('Centrality', fontsize=24, y=0.93)
# plt.savefig(f'Figures/{name}_centrality_world_new4.pdf', dpi=1000)
plt.savefig(f'Figures/centralities_world_12x8.pdf', dpi=600)


## Degree Distribution

In [ ]:
G_DEGS = [GG.degree()[i] for i in GG.nodes()]
bins = np.logspace(np.log10(min(G_DEGS)), np.log10(max(G_DEGS) + 1), 30)
# take the average value of all degrees
AVG_VAL = np.average(G_DEGS)

In [ ]:
fig = plt.figure(dpi=300, figsize=(6, 6))
plt.hist(G_DEGS, bins=bins, density=True, alpha=0.8)
plt.axvline(x=AVG_VAL, color='r', label='Average Degree', c='orange')
plt.xlabel('Degree $k$', fontsize=14)
plt.ylabel('$\mathcal{P}(k)$', fontsize=14)
plt.title('Degree Distribution', fontsize=20)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.savefig('Figures/degree_distribution.pdf', dpi=1000)
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(8, 4))
plt.plot(degree_centrality.keys(), degree_centrality.values(), linewidth=0, marker='.', label='Degree Centrality',alpha=0.3)
plt.plot(closeness_centrality.keys(), closeness_centrality.values(), linewidth=0, marker='.',
label='Closeness Centrality', alpha=0.3)
plt.plot(betweenness_centrality.keys(), betweenness_centrality.values(), linewidth=0, marker='.',
label='Betweenness Centrality', alpha=0.3)
plt.plot(eigenvector_centrality.keys(), eigenvector_centrality.values(), linewidth=0, marker='.',
label='Eigenvector Centrality', alpha=0.3)
plt.title('Centralities', fontsize=20)
ax.xaxis.set_major_locator(plt.MaxNLocator(3))
plt.xlabel('Airport ID $n$')
plt.ylabel('Centrality value')
plt.legend()
plt.savefig('Figures/centralities.pdf', dpi=1000)
plt.show()

build a dataframe with rows as airport ids and columns as the centralities.
This allows us to use seaborn's pair plotting feature.

In [ ]:
centralityDict = {}
for k in GG.nodes():
    centralityDict[k] = {}
    centralityDict[k]['betweenness_centrality'] = betweenness_centrality[k]
    centralityDict[k]['closeness_centrality'] = closeness_centrality[k]
    centralityDict[k]['degree_centrality'] = degree_centrality[k]
    centralityDict[k]['eigenvector_centrality'] = eigenvector_centrality[k]
df_centralities = pd.DataFrame.from_dict(centralityDict, orient='index')

In [ ]:
fig = plt.figure(dpi=100, figsize=(10,10))
f = sns.pairplot(df_centralities,
                 # kind='hist',
                 plot_kws={
    'alpha':0.6,
    # 'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}
}, diag_kind = 'kde'
                 )
f.fig.suptitle('Centralities', y=1.04, fontsize=24)
plt.savefig('Figures/centrality_pairplot.pdf', dpi=100)

Get degrees for each node. We want to find the airports with the highest degrees.

In [ ]:
degrees = nx.degree(GG)

In [ ]:
# un-normalized betweenness centrality. just out of curiosity. not used.
# betweenness_centrality_unnormalized = nx.betweenness_centrality(GG, normalized=False)
# print(min(betweenness_centrality.values()), max(betweenness_centrality.values()))
# print(min(betweenness_centrality_unnormalized.values()), max(betweenness_centrality_unnormalized.values()))

In [ ]:
# sort betweenness centrality to take highest values
# sortedByBetweennessCentrality = dict(sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True))
# df_betweennesscentrality = df_centralities.nlargest(n=10, columns='betweenness_centrality')

In [ ]:
df_centralities.index.name = 'Airport ID'
df_airports.set_index(['Airport ID'], inplace=True)
df_airports.index = df_airports.index.astype(int)
df_centralities.index = df_centralities.index.astype(int)

In [ ]:
print(df_centralities.dtypes)
print(df_airports.dtypes)

df_1 = df_centralities.join(df_airports, how='left')

In [ ]:
df_1 = df_1.nlargest(n=10, columns='betweenness_centrality')
df_1.to_csv('airports_by_betweenness_centr.csv')

In [ ]:
fig=plt.figure()

bins = np.logspace(np.log10(min(betweenness_centrality.values())), np.log10(max(betweenness_centrality.values()) + 1), 30)
plt.hist(betweenness_centrality.values(), bins=25, density=True)
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
# G_DEGS = [GG.degree()[i] for i in GG.nodes()]
# sns.kdeplot(G_DEGS)